In [ ]:
"""
GGUF Downloader (HF Hub -> local / Google Drive)
------------------------------------------------
This script helps you:
  1) Log in to Hugging Face (token required for gated/private repos)
  2) Select a model repo that contains **.gguf** files
  3) List available .gguf files
  4) Download a chosen file to:
       - Colab: /content/drive/MyDrive/gguf  (Google Drive mounted)
       - Local: ~/gguf  (or a custom folder you set below)

It runs in two modes automatically:
  • Colab with widgets (Dropdowns & Buttons)
  • CLI/Local fallback (terminal prompts in PyCharm, VSCode, etc.)

Tip: If your base repo (e.g. `janhq/Jan-v1-4B`) does not offer GGUF files,
use a GGUF mirror like `TheBloke/...-GGUF` or `bartowski/...-GGUF` and paste that Repo ID.
"""


In [ ]:
import os
import sys
import re
import json
import pathlib
import traceback
from typing import List, Optional

# Detect Colab
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# Best-effort install missing packages if in Colab
def _ensure(pkgs: List[str]):
    if not IN_COLAB:
        return
    import importlib, subprocess
    missing = []
    for p in pkgs:
        try:
            importlib.import_module(p.split("==")[0].split(">=")[0])
        except Exception:
            missing.append(p)
    if missing:
        print("Installing:", missing)
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + missing)

_ensure(["huggingface_hub>=0.23.0", "ipywidgets>=8.0.0", "tqdm"])  # tqdm used by hub utilities

from getpass import getpass
from huggingface_hub import (
    HfApi,
    login as hf_login,
    notebook_login as hf_notebook_login,
    whoami,
    hf_hub_download,
    HfHubHTTPError,
)

# Widgets only if Colab
if IN_COLAB:
    import ipywidgets as w
    from IPython.display import display, clear_output


In [ ]:
# Preset repos that commonly provide GGUF variants. These may change over time; paste your own repo if needed.
PRESETS = {
    "— Choose a preset —": "",
    "Jan v1 4B (base repo, may not host GGUF)": "janhq/Jan-v1-4B",
    "TinyLlama 1.1B Chat (GGUF by TheBloke)": "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF",
    "Mistral 7B Instruct v0.2 (GGUF by TheBloke)": "TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
    "Llama 3 8B Instruct (GGUF by bartowski)": "bartowski/Llama-3-8B-Instruct-GGUF",
    "Phi-2 (GGUF by TheBloke)": "TheBloke/phi-2-GGUF",
    "Custom (paste below)": "__CUSTOM__",
}

# Default output directories
if IN_COLAB:
    DEFAULT_OUT_DIR = "/content/drive/MyDrive/gguf"
else:
    DEFAULT_OUT_DIR = os.path.expanduser("~/gguf")


In [ ]:
api = HfApi()


def ensure_login() -> None:
    """Make sure we're logged in to the HF Hub (if needed)."""
    try:
        info = whoami()
        if info and isinstance(info, dict):
            print(f"✔ Already logged in as: {info.get('name') or info.get('username')}")
            return
    except Exception:
        pass

    token = os.getenv("HF_TOKEN")
    if token:
        try:
            hf_login(token=token)
            print("✔ Logged in using HF_TOKEN env var")
            return
        except Exception as e:
            print("Env token login failed:", e)

    if IN_COLAB:
        print("No token found. Opening Colab login widget…")
        print("Hint: You can also set HF_TOKEN in Colab secrets or environment variables for reuse across sessions.")
        hf_notebook_login()
    else:
        print("No token found. You can set HF_TOKEN env var or paste manually.")
        print("Hint: For persistence, you can export HF_TOKEN in your shell profile.")
        tk = getpass("Paste your Hugging Face token: ")
        if tk.strip():
            hf_login(token=tk.strip())
            print("✔ Logged in")


def list_gguf_files(repo_id: str, revision: Optional[str] = None) -> List[str]:
    """Return a list of .gguf files in the given repo (top-level or nested)."""
    try:
        files = api.list_repo_files(repo_id=repo_id, revision=revision)
    except HfHubHTTPError as e:
        print("Hub error while listing files:", e)
        return []
    except Exception as e:
        print("Unexpected error:", e)
        return []
    ggufs = [f for f in files if f.lower().endswith(".gguf")]
    ggufs.sort()
    return ggufs


def download_gguf(repo_id: str, filename: str, out_dir: str, revision: Optional[str] = None) -> str:
    """Download a single GGUF file to out_dir and return the local path."""
    os.makedirs(out_dir, exist_ok=True)
    print(f"Downloading {filename} from {repo_id} → {out_dir}")
    try:
        local_path = hf_hub_download(
            repo_id=repo_id,
            filename=filename,
            revision=revision,
            local_dir=out_dir,
            local_dir_use_symlinks=False,
            force_download=False,
        )
        print("✔ Done:", local_path)
        return local_path
    except HfHubHTTPError as e:
        print("Hub error during download:", e)
        raise



In [ ]:

def run_colab_ui():
    if not IN_COLAB:
        return False

    # Mount Drive if available
    try:
        from google.colab import drive  # type: ignore
        drive.mount("/content/drive")
        print("✔ Google Drive mounted.")
    except Exception:
        print("(Drive mount skipped)")

    ensure_login()

    preset_dd = w.Dropdown(options=list(PRESETS.keys()), value="— Choose a preset —", description="Preset:")
    custom_repo = w.Text(placeholder="e.g. TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF", description="Repo:")
    rev_text = w.Text(placeholder="optional (e.g. main)", description="Revision:")
    out_dir = w.Text(value=DEFAULT_OUT_DIR, description="Out dir:")
    list_btn = w.Button(description="List GGUF files", button_style="info")
    files_dd = w.Dropdown(options=["(list files…)"], description="File:")
    dl_btn = w.Button(description="Download", button_style="success")
    log_out = w.Output()

    def _resolve_repo(change=None):
        key = preset_dd.value
        repo = PRESETS.get(key, "")
        if repo == "__CUSTOM__":
            repo = custom_repo.value.strip()
        return repo

    def on_list_clicked(_):
        with log_out:
            clear_output()
            repo = _resolve_repo()
            if not repo:
                print("Please choose a preset or paste a repo ID.")
                return
            print("Fetching file list from:", repo)
            files = list_gguf_files(repo_id=repo, revision=rev_text.value.strip() or None)
            if not files:
                print("No .gguf files found. Is this repo a GGUF mirror? Try a different preset or paste a GGUF repo.")
                return
            files_dd.options = files
            print(f"Found {len(files)} GGUF files.")

    def on_download_clicked(_):
        with log_out:
            clear_output()
            repo = _resolve_repo()
            if not repo:
                print("Please choose a preset or paste a repo ID.")
                return
            filename = files_dd.value
            if not filename or filename.startswith("("):
                print("Please list and select a file first.")
                return
            try:
                path = download_gguf(
                    repo_id=repo,
                    filename=filename,
                    out_dir=out_dir.value.strip() or DEFAULT_OUT_DIR,
                    revision=rev_text.value.strip() or None,
                )
                size_mb = os.path.getsize(path) / (1024 * 1024)
                print(f"Saved: {path}  ({size_mb:.1f} MB)")
            except Exception:
                traceback.print_exc()

    list_btn.on_click(on_list_clicked)
    dl_btn.on_click(on_download_clicked)

    ui = w.VBox([
        preset_dd,
        custom_repo,
        rev_text,
        out_dir,
        w.HBox([list_btn, dl_btn]),
        files_dd,
        log_out,
    ])
    display(ui)
    return True



In [ ]:

def run_cli():
    if IN_COLAB:
        return False

    ensure_login()

    print("\n=== GGUF Downloader (CLI) ===")
    print("Presets:")
    for i, k in enumerate(PRESETS.keys()):
        print(f"  [{i}] {k}")
    try:
        idx = int(input("Select preset index (or any other key for custom): ") or "0")
    except Exception:
        idx = 0
    keys = list(PRESETS.keys())
    preset = keys[idx] if 0 <= idx < len(keys) else keys[0]

    if PRESETS[preset] in ("", "__CUSTOM__"):
        repo_id = input("Paste GGUF repo id (e.g. TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF): ").strip()
    else:
        repo_id = PRESETS[preset]
    if not repo_id:
        print("No repo provided. Exiting.")
        return True

    revision = input("Optional revision (press Enter for default): ").strip() or None
    out_dir = input(f"Output dir [{DEFAULT_OUT_DIR}]: ").strip() or DEFAULT_OUT_DIR

    files = list_gguf_files(repo_id, revision)
    if not files:
        print("No GGUF files found in this repo.")
        return True

    print("\nAvailable GGUF files:")
    for i, f in enumerate(files):
        print(f"  [{i}] {f}")
    try:
        fidx = int(input("Pick file index to download: "))
    except Exception:
        print("Invalid selection.")
        return True

    if fidx < 0 or fidx >= len(files):
        print("Index out of range.")
        return True

    filename = files[fidx]
    path = download_gguf(repo_id, filename, out_dir, revision)
    size_mb = os.path.getsize(path) / (1024 * 1024)
    print(f"Saved: {path}  ({size_mb:.1f} MB)")
    return True



In [ ]:
if __name__ == "__main__":
    ran = False
    try:
        if IN_COLAB:
            # Try to run the Colab widget UI
            ran = run_colab_ui()
    except Exception:
        print("Widget UI failed; falling back to CLI…")
        traceback.print_exc()

    if not ran:
        # Fallback to CLI/Local mode
        run_cli()